# Question 1: Transforming Wikipedia Table to a Python Dataframe

#### Install BeautifulSoup4, requests, lxml

In [5]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.9.0-py36_0      anaconda
    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2020.4.5.1-py36_0 --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1g-

In [3]:
!conda install -c anaconda requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    requests-2.23.0            |           py36_0          91 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2020.4.5.1-py36_0 --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1g-h7b6447c_0 --> 1.1.1g-h7b6447c_0 anaconda
    requests:        2.21.0-py36_0

In [9]:
!conda install -c anaconda lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB  anaconda

The following packages will be UPDATED:

    lxml: 4.3.1-py36hefd8a0e_0 --> 4.5.0-py36hefd8a0e_0 anaconda


lxml-4.5.0           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


#### Import libraries

In [10]:
from bs4 import BeautifulSoup
import requests
import lxml.html
import pandas as pd

#### Use BeautifulSoup to parse the table from wiki webpage

In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('table',class_ = 'wikitable')
rows = table.find_all('tr')

#### Make a dataframe

In [12]:
data = []

for tr in rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    data.append(row)
    
df = pd.DataFrame(data, columns = ['Postal Code','Borough','Neighborhood'])  # add column headers to the dataframe

df = df.drop(df.index[0]) # remove the first row with "none"

#### remove "Not assigned" cells in column Borough

In [13]:
for i in range(1,181):
    if df.loc[i,'Borough'] == 'Not assigned':
        df.drop([i],inplace = True)
df_clean = df
df_clean.head(15)

,Postal Code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"
12,M3B,North York,Don Mills
13,M4B,East York,"Parkview Hill, Woodbine Gardens"
14,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Count dataframe rows by .shape function

In [14]:
df_clean.shape

(103, 3)

# Question 2: Adding Geographical Coordinates

In [13]:
! pip install geocoder

     |████████████████████████████████| 102kB 7.8MB/s ta 0:00:011


In [7]:
import geocoder

In [39]:
g = geocoder.google('Fairfax, Virginia')
print(g.latlng)

None


In [ ]:
lat_lng_coords = None
while(lat_lng_coords is None):
    g = geocoder.google('{}, Fairfax, VA'.format(df_clean['Postal Code']))
    lat_lng_coords = g.latlng

print(Lat_lng_coords)

#### The above cells confirmed that the Geocoder package wasn't working
#### therefore using the csv file instead

In [15]:
df_coords = pd.read_csv('http://cocl.us/Geospatial_data')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
for i in range(103):
    if df_coords.loc[i, 'Postal Code'] == M5V:
        print(df_coords.loc[i, 'Latitude'], df_coords.loc[i, 'Longitutde'])

NameError: name 'M5V' is not defined

#### combine the two dataframe
#### Step 1: make index of two dataframes consistent 

In [28]:
df_clean.reset_index(drop = True, inplace = True)
df_clean.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Step 2: get Latitude and Longitude into the dataframe

In [29]:
Latitude = []
Longitude = []

for a in range(103):
    for b in range(103):
        if df_clean.loc[a, 'Postal Code'] == df_coords.loc[b, 'Postal Code']:
            Latitude.append(df_coords.loc[b, 'Latitude'])
            Longitude.append(df_coords.loc[b, 'Longitude'])
            
df_clean['Latitude'] = Latitude
df_clean['Longitude'] = Longitude
df_clean

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Question 3: Cluster the Neighborhoods in Toronto

In [31]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

In [32]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Select only boroughs that contain the word Toronto

In [43]:
df_Toronto = df_clean[df_clean['Borough'].str.contains('Toronto')]
df_Toronto.reset_index(drop = True, inplace = True)
df_Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### Group neighborhoods in Toronto by Borough

In [47]:
df_Toronto_grouped = df_Toronto.groupby('Borough').mean().reset_index()
df_Toronto_grouped

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654597,-79.383972
2,East Toronto,43.669436,-79.324654
3,West Toronto,43.652653,-79.449290


#### Cluster neighborhoods of Toronto

In [69]:
kclusters = 4
Toronto_grouped_clustering = df_Toronto_grouped.drop('Borough', 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(Toronto_grouped_clustering)
kmeans.labels_[0:10]

array([0, 3, 1, 2], dtype=int32)

In [71]:
df_Toronto_grouped.insert(3, 'Cluster Labels', kmeans.labels_)
df_Toronto_grouped

,Borough,Latitude,Longitude,Cluster Labels
0,Central Toronto,43.701980,-79.398954,0
1,Downtown Toronto,43.654597,-79.383972,3
2,East Toronto,43.669436,-79.324654,1
3,West Toronto,43.652653,-79.449290,2


#### Get Toronto's geograpical coordinate

In [52]:
from geopy.geocoders import Nominatim

In [54]:
address = 'Toronto'

geolocator = Nominatim(user_agent = 'T_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are ', latitude, longitude)

The geograpical coordinate of Toronto are  43.6534817 -79.3839347


#### Visulize the clusters

In [55]:
import numpy as np

In [73]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df_Toronto_grouped['Latitude'], df_Toronto_grouped['Longitude'],
                                 df_Toronto_grouped['Borough'], df_Toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
    [lat, lon],
    radius = 5,
    popup = label,
    color = rainbow[cluster - 1],
    fill = True,
    fill_color = rainbow[cluster - 1],
    fill_opacity = 0.7).add_to(map_clusters)

map_clusters

In [76]:
!jupyter trust mynotebook.ipynb

[TrustNotebookApp] ERROR | Exception while loading config file /var/pod/.runtime/ax-ext/config/wscloud/jupyter_notebook_config.py
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/traitlets/config/application.py", line 562, in _load_config_files
    config = loader.load_config()
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/traitlets/config/loader.py", line 457, in load_config
    self._read_file_as_dict()
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/traitlets/config/loader.py", line 489, in _read_file_as_dict
    py3compat.execfile(conf_filename, namespace)
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ipython_genutils/py3compat.py", line 198, in execfile
    exec(compiler(f.read(), fname, 'exec'), glob, loc)
  File "/var/pod/.runtime/ax-ext/config/wscloud/jupyter_notebook_config.py", line 17, in <module>
    from cdsax_jupyter_extensions.ax_log import ax_log_request
ModuleNotFoundError: No mod